In [ ]:
#1 fake news detection


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [ ]:
df_fake=pd.read_csv("/content/Fake.csv")
df_real=pd.read_csv("/content/True.csv")

In [ ]:
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
df_real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
df_fake["class"]=0
df_real["class"]=1

In [ ]:
df_fake.shape,df_real.shape
# df_real.shape

((23481, 5), (21417, 5))

In [ ]:
#dta for manual testing
df_fake_manual_testing=df_fake.tail(10)
for i in range(23480,23470,-1):
  df_fake.drop([i],axis=0,inplace=True)
df_real_manual_testing=df_real.tail(10)
for i in range(21416,21406,-1):
  df_real.drop([i],axis=0,inplace=True)

In [ ]:
df_manual_testing=pd.concat( [df_fake_manual_testing,df_real_manual_testing],axis=0)
df_manual_testing.to_csv("manual_testing.csv")


In [ ]:
df_merge=pd.concat([df_fake,df_real],axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
#removing unwanted columns like subject date
df= df_merge.drop(["title","subject","date"],axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [ ]:
#sampling the data
df=df.sample(frac=1)
df.head(10)

,text,class
18256,"LOS TEQUES, Venezuela (Reuters) - Venezuelan s...",1
6112,"A group of about 1,000 Christians planned to a...",0
16476,How unfunny! Does this not tell you something ...,0
4577,It s no secret that Donald Trump is cruel to w...,0
22856,"21st Century WireYesterday, US mainstream medi...",0
1056,Alleged President Donald Trump has been on the...,0
17732,BELGRADE (Reuters) - Catalonia s president has...,1
19241,"On March 31, 2015 Sergeant Andrew Tahmooressi ...",0
21227,GENEVA (Reuters) - Civilians caught up in the ...,1
17135,AMSTERDAM (Reuters) - The U.N. Yugoslav tribun...,1


In [ ]:
#data transformation
#checking null values
df.isnull().sum()

text     0
class    0
dtype: int64

In [ ]:
#removing unwanted and special characters
def wordopt(text):
  text=text.lower()
  text=re.sub('\[.*?\]', '', text)
  text=re.sub("\\W"," ",text)
  text=re.sub('https?://\S+|www\.\S+', '', text)
  text=re.sub('<.*?>+', '', text)
  text=re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text=re.sub('\n', '', text)
  return re.sub('\w*\d\w*', '', text)


In [ ]:
df["text"]=df["text"].apply(wordopt)

In [ ]:
df.head(10)

,text,class
18256,los teques venezuela reuters venezuelan s...,1
6112,a group of about christians planned to atten...,0
16476,how unfunny does this not tell you something ...,0
4577,it s no secret that donald trump is cruel to w...,0
22856,century wireyesterday us mainstream media ou...,0
1056,alleged president donald trump has been on the...,0
17732,belgrade reuters catalonia s president has...,1
19241,on march sergeant andrew tahmooressi was dr...,0
21227,geneva reuters civilians caught up in the ...,1
17135,amsterdam reuters the u n yugoslav tribun...,1


In [ ]:
x=df["text"]
y=df["class"]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorize=TfidfVectorizer()
xvector_train=vectorize.fit_transform(x_train)
xvector_test=vectorize.transform(x_test)

**Logistic Regression**


In [ ]:
#importing logistic regression
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(xvector_train,y_train)
pred_lr=lr.predict(xvector_test)


In [ ]:
Score=lr.score(xvector_test,y_test)
Score

0.9870766488413547

In [ ]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4681
           1       0.99      0.99      0.99      4295

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



**Decision Tree Classification**

In [ ]:
#importing Decision tree
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(xvector_train,y_train)
pred_dt=dt.predict(xvector_test)

In [ ]:
Score=dt.score(xvector_test,y_test)
Score

0.9959893048128342

In [ ]:
print(classification_report(y_test,pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4681
           1       1.00      0.99      1.00      4295

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



**Gradient Boosting Classifier**

In [27]:
#import boosting classifier\
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(xvector_train,y_train)
pred_gb=gb.predict(xvector_test)

In [28]:
Score=gb.score(xvector_test,y_test)
Score

0.9957664884135472

In [29]:
print(classification_report(y_test,pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4681
           1       0.99      1.00      1.00      4295

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



**Random Forest**

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(xvector_train,y_train)
pred_rf=rf.predict(xvector_test)

In [31]:
Score=rf.score(xvector_test,y_test)
Score

0.9904188948306596

In [ ]:
print(classification_report(y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4656
           1       0.99      0.99      0.99      4320

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [32]:
#Manual Testing
def output_label(n):
  if n==0:
    return "Fake News"
  elif n==1:
    return "Real News"
def manual_testing(news):
  Testing_news={"test":[news]}
  new_def_test=pd.DataFrame(Testing_news)
  new_def_test["test"]=new_def_test["test"].apply(wordopt)
  new_x_test=new_def_test["test"]
  new_xv_test=vectorize.transform(new_x_test)
  pred_rf=rf.predict(new_xv_test)
  pred_lr=lr.predict(new_xv_test)
  pred_dt=dt.predict(new_xv_test)
  pred_gb=gb.predict(new_xv_test)

  return print("\n\nLinear Regression predictions:{}\n\nDecision Tree predictions:{}\n\nRandom Forest predictions:{}\n\nGradient Boosting predictions:  {}".format(output_label(pred_lr[0]),output_label(pred_dt[0]),output_label(pred_rf[0]),output_label(pred_gb[0])))

In [ ]:
news=str(input())
manual_testing(news)